In [2]:
# import pandas as pd
# from transformers import pipeline
# from transformers import AutoModelForSequenceClassification
# import torch
# from torch.utils.data import DataLoader
# from transformers import Trainer, TrainingArguments,AutoTokenizer
# from sklearn.preprocessing import StandardScaler
# from sklearn.model_selection import train_test_split
# from torch.utils.data import DataLoader

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline
import gradio as gr
import numpy as np
import tensorflow as tf

In [2]:
# Step 1: Define a class or function for interacting with the model
class ConversationModel:
    def __init__(self, model_name="m42-health/Llama3-Med42-8B"):
        # Load the model once when the class is initialized
        self.pipe = pipeline("text-generation", model=model_name)

    def ask(self, messages, max_new_tokens=200):
        # Send the messages to the model and return the response
        return self.pipe(messages, max_length=2048, do_sample=True)

In [ ]:
model_instance = ConversationModel()

In [ ]:
messages = [{"role": "user", "content": "what is diabetes"}]
response = model_instance.ask(messages)


In [ ]:
print(response)

In [1]:
import gradio as gr
import numpy as np
import os
from model import main, load_trained_model, preprocess_input, predict

# Create Model
if not os.path.exists('diabetes_model.h5'):
    main()
else:
    print('Model already exists')

# Load your trained model
model = load_trained_model('diabetes_model.h5')

# Prediction function
def predict_diabetes(HighBP, CholCheck, BMI, Smoker, Stroke, HeartDiseaseorAttack,
                     PhysActivity, Fruits, Veggies, HvyAlcoholConsump, GenHlth,
                     MentHlth, PhysHlth, DiffWalk, Sex, Age, Education, Income,
                     AnyHealthcare, NoDocbcCost):

# Collect user input data into a list
    user_data = [HighBP, CholCheck, BMI, Smoker, Stroke, HeartDiseaseorAttack, PhysActivity,
                 Fruits, Veggies, HvyAlcoholConsump, GenHlth, MentHlth, PhysHlth, DiffWalk,
                 Sex, Age, Education, Income, AnyHealthcare, NoDocbcCost]

    # Ensure user_data is a 2D numpy array with shape (1, n)
    user_data_np = np.array([user_data])  # Convert to 2D numpy array for model input

    # Preprocess the input if needed (optional if your model expects raw inputs directly)
    processed_data = preprocess_input(user_data_np)

    # Predict using the loaded model
    prediction = predict(processed_data, model)

    return prediction

# Define the Gradio interface
interface = gr.Interface(
    fn=predict_diabetes,  # Function to call when inputs are submitted
    inputs=[
        gr.components.Dropdown([0, 1], label="High Blood Pressure (HighBP)"),
        gr.components.Dropdown([0, 1], label="Cholesterol Check (CholCheck)"),
        gr.components.Slider(10, 50, step=1, label="BMI"),
        gr.components.Dropdown([0, 1], label="Smoker"),
        gr.components.Dropdown([0, 1], label="Stroke"),
        gr.components.Dropdown([0, 1], label="Heart Disease or Attack"),
        gr.components.Dropdown([0, 1], label="Physical Activity (PhysActivity)"),
        gr.components.Dropdown([0, 1], label="Eats Fruits"),
        gr.components.Dropdown([0, 1], label="Eats Veggies"),
        gr.components.Dropdown([0, 1], label="Heavy Alcohol Consumption"),
        gr.components.Slider(1, 5, step=1, label="General Health (GenHlth)"),
        gr.components.Slider(0, 30, step=1, label="Mental Health Days (MentHlth)"),
        gr.components.Slider(0, 30, step=1, label="Physical Health Days (PhysHlth)"),
        gr.components.Dropdown([0, 1], label="Difficulty Walking (DiffWalk)"),
        gr.components.Dropdown([0, 1], label="Sex (0=Female, 1=Male)"),
        gr.components.Slider(18, 80, step=1, label="Age"),
        gr.components.Slider(1, 12, step=1, label="Education Level"),
        gr.components.Slider(1, 8, step=1, label="Income Level"),
        gr.components.Dropdown([0, 1], label="Any Healthcare Access"),
        gr.components.Dropdown([0, 1], label="No Doctor Because of Cost"),
    ],
    outputs=gr.Textbox(label="Prediction"),  # Output will be a text prediction
    title="Diabetes Prediction",
    description="Enter your information to predict the risk of diabetes. (0=no 1=yes)",
)

# Launch the Gradio interface
interface.launch()


Model already exists
* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
